# Process data for dynamic_relax calculation

This Notebook is designed for reading and converting results from the dynamic_relax calculation

Library imports

In [30]:
from __future__ import print_function, division

import glob
import os
from collections import OrderedDict
from datetime import date
from math import floor
from copy import deepcopy

import pandas as pd
pd.options.display.max_rows = 50

import numpy as np

from DataModelDict import DataModelDict as DM

import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

import iprPy

In [2]:
from bokeh.plotting import figure, output_file, show
from bokeh.embed import components
from bokeh.resources import Resources
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## 1. Read Raw Data

This section reads in raw data from a database 

### 1.1 Initialize database

- __dbasename__ is used here to predefine different dbase settings
- __dbase__ is the iprPy.Database object to use for accessing a database

In [3]:
dbasename = 'local'

# 'local' is a local directory
if   dbasename == 'local':
    dbase = iprPy.Database('local',   host='C:\Users\lmh1\Documents\calculations\ipr\library')

# 'curator' is a local MDCS curator
elif dbasename == 'curator':
    dbase = iprPy.Database('curator', host='http://127.0.0.1:8000/', 
                                      user='admin', 
                                      pswd='admin')

# 'iprhub' is the remote MDCS curator at iprhub
elif dbasename == 'iprhub':
    dbase = iprPy.Database('curator', host='https://iprhub.nist.gov/', 
                                      user='lmh1',
                                      pswd='C:/users/lmh1/documents/iprhub/iprhub_password.txt',
                                      cert='C:/users/lmh1/documents/iprhub/iprhub-ca.pem')
else:
    raise ValueError('unknown dbasename ' + dbasename)

### 1.2 Access records

- __dynamic_relax_recordstyle__ is the iprPy.Record style associated with the dynamic_relax simulations

- __prototype_recordstyle__ is the iprPy.Record style associated with the crystal prototype definintions

In [4]:
dynamic_relax_recordstyle = 'calculation-dynamic-relax'
prototype_recordstyle = 'crystal-prototype'

Access the prototype records

- __proto_df__ is a DataFrame of the prototype records

In [5]:
proto_df = []
for record in dbase.iget_records(style=prototype_recordstyle):
    proto_df.append(record.todict())
proto_df = pd.DataFrame(proto_df)
print(str(len(proto_df)) + ' records loaded')

19 records loaded


Access the calculation records

- __raw_df__ is a DataFrame of the raw calculation records

In [6]:
raw_df = []
records = None
for i in xrange(10):
    try:
        records = dbase.get_records(style=dynamic_relax_recordstyle)
    except:
        print('failed '+ str(i+1) + ' times')

if records is not None:        
    for record in records:
        raw_df.append(record.todict())
    raw_df = pd.DataFrame(raw_df)
else:
    raise ValueError('Failed to load')
print(str(len(raw_df)) + ' records loaded')

7677 records loaded


### 1.3 Check errors

In [7]:
if 'error' in raw_df:
    for error in np.unique(raw_df[pd.notnull(raw_df.error)].error):
        print(error.split('\n')[-1])

ERROR on proc 0: Too many neighbor bins (../neighbor.cpp:1854)
ERROR on proc 5: Too many neighbor bins (../neighbor.cpp:1854)
ERROR on proc 0: Too many neighbor bins (../neighbor.cpp:1854)
pymbar.utils.ParameterError: Sample covariance sigma_AB^2 = 0 -- cannot compute statistical inefficiency
ERROR: Lost atoms: original 12000 current 11999 (../thermo.cpp:427)
pymbar.utils.ParameterError: Sample covariance sigma_AB^2 = 0 -- cannot compute statistical inefficiency
ERROR: Lost atoms: original 2000 current 1998 (../thermo.cpp:427)
ERROR: Lost atoms: original 16000 current 15999 (../thermo.cpp:427)
pymbar.utils.ParameterError: Sample covariance sigma_AB^2 = 0 -- cannot compute statistical inefficiency
ERROR: Domain too large for neighbor bins (../neighbor.cpp:1772)
pymbar.utils.ParameterError: Sample covariance sigma_AB^2 = 0 -- cannot compute statistical inefficiency
ERROR: Lost atoms: original 4000 current 3998 (../thermo.cpp:427)
ERROR: Lost atoms: original 16000 current 15996 (../thermo

## 2. Process Data

This section processes and refines the data

### Remove error data

In [8]:
raw_df = raw_df[pd.isnull(raw_df.error)]
print(str(len(raw_df)) + ' suscessful records')

6989 suscessful records


### 2.1 Identify composition

- __counts__ is a dictionary counting the number of times each atype appears in a crystal prototype's unit cell (i.e. the number of symmetry equivalent sites)

In [9]:
counts = {}
for i, prototype in proto_df.iterrows():
    model = DM(dbase.get_record(name=prototype.id).content)
    counts[prototype.id] = np.unique(model.finds('component'), return_counts=True)[1]

- __comp_refine()__ takes a list of symbols and count of how many times each symbol appears in a structure and generates a composition string.

In [32]:
def comp_refine(symbols, counts):
    """Takes a list of symbols and count of how many times each symbol appears and generates a composition string."""
    primes = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47]
    
    sym_dict = {}
    for i in xrange(len(symbols)):
        sym_dict[symbols[i]] = counts[i]
    
    for prime in primes:
        if max(sym_dict.values()) < prime:
            break
        
        while True:
            breaktime = False
            for value in sym_dict.values():
                if value % prime != 0:
                    breaktime = True
                    break
            if breaktime:
                break
            for key in sym_dict:
                sym_dict[key] //= prime
    
    composition=''
    for key in sorted(sym_dict):
        if sym_dict[key] > 0:
            composition += key
            if sym_dict[key] != 1:
                composition += str(sym_dict[key])
            
    return composition       

In [35]:
compositions = []
for i, calc in raw_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.prototype]))
raw_df = raw_df.assign(composition=compositions)

### 2.2 Remove duplicates and meaningless entries

- __df__ is the dataframe during/after processing and refining

In [36]:
#Ignore any that don't use the standard run parameters
df = raw_df[np.isclose(raw_df.temperature, 0.0) & 
            np.isclose(raw_df.pressure_xx, 0.0) & 
            np.isclose(raw_df.pressure_yy, 0.0) & 
            np.isclose(raw_df.pressure_zz, 0.0)]
print(len(df), '0 K, 0 pressure runs')

#Ignore false compounds (ones where number of unique symbols != number of symbols)
df = df[df.symbols.apply(lambda x: len(np.unique(x))) == df.symbols.apply(lambda x: len(x))] 
print(len(df), 'real compounds')

#Ignore duplicate compounds
ignore = set()
for i in xrange(len(df)):
    trunc = df.iloc[i+1:]
    matches = trunc.calc_key[(trunc.potential_id == df.iloc[i].potential_id) & 
                             (trunc.prototype ==    df.iloc[i].prototype) &
                             (trunc.composition ==  df.iloc[i].composition) ].tolist()
    ignore = ignore.union(matches)
df = df[~df.calc_key.isin(ignore)]
print(len(df), 'non-duplicates')

df.reset_index(drop=True, inplace=True)
print(str(len(df)) + ' records after filtering')

6989 0 K, 0 pressure runs
4320 real compounds
2972 non-duplicates
2972 records after filtering


### 2.3 Remove structures that have relaxed to another crystal family

- __processed_df__ is all records after the above filtering

In [37]:
processed_df = deepcopy(df)

Sort by prototype's (orthorhombic) crystal family

- __cubic_df__ is all records that have a cubic prototype
- __hexagonal_df__ is all records that have a hexagonal prototype
- __tetragonal_df__ is all records that have a tetragonal prototype
- __orthorhombic_df__ is all records that have a orthorhombic prototype

In [38]:
cubicprotos = []
hexagonalprotos = []
tetragonalprotos = []
orthorhombicprotos = []

for i, proto in proto_df.iterrows():
    if   np.isclose(proto.c, 1.0, atol=0, rtol=1e-8):    cubicprotos.append(proto.id)
    elif np.isclose(proto.b, 1.0, atol=0, rtol=1e-8):    tetragonalprotos.append(proto.id)
    elif np.isclose(proto.b, 3**0.5, atol=0, rtol=1e-8): hexagonalprotos.append(proto.id)
    else:                                                orthorhombicprotos.append(proto.id)       

cubic_df =        df[df.prototype.isin(cubicprotos)] 
hexagonal_df =    df[df.prototype.isin(hexagonalprotos)] 
tetragonal_df =   df[df.prototype.isin(tetragonalprotos)] 
orthorhombic_df = df[df.prototype.isin(orthorhombicprotos)] 
        
print(len(cubic_df), 'cubic structures')
print(len(hexagonal_df), 'hexagonal structures')
print(len(tetragonal_df), 'tetragonal structures')
print(len(orthorhombic_df), 'orthorhombic structures')

1985 cubic structures
539 hexagonal structures
448 tetragonal structures
0 orthorhombic structures


Filter out cubic structures that are now non-cubic

In [39]:
df = df[~df.calc_key.isin(cubic_df.calc_key) | 
        (df.calc_key.isin(cubic_df.calc_key) & np.isclose(df.a_mean, df.b_mean, rtol=0, atol=3*df.a_std)
                                             & np.isclose(df.a_mean, df.c_mean, rtol=0, atol=3*df.a_std))]
print(len(df))

2246


Filter out hexagonal structures that are now not hexagonal

In [40]:
df = df[~df.calc_key.isin(hexagonal_df.calc_key) | 
        (df.calc_key.isin(hexagonal_df.calc_key) & np.isclose(df.a_mean, df.b_mean/(3**0.5), rtol=0, atol=3*df.a_std))]
print(len(df))

2200


Filter out tetragonal structures that are now not tetragonal

In [41]:
df = df[~df.calc_key.isin(tetragonal_df.calc_key) | 
        (df.calc_key.isin(tetragonal_df.calc_key) &  np.isclose(df.a_mean, df.b_mean, rtol=0, atol=3*df.a_std)
                                                  & ~np.isclose(df.a_mean, df.c_mean, rtol=0, atol=3*df.a_std))]
print(len(df))

2019


Filter out orthorhombic structures that are now not orthorhombic

In [42]:
df = df[~df.calc_key.isin(orthorhombic_df.calc_key) | 
        (df.calc_key.isin(orthorhombic_df.calc_key) & ~np.isclose(df.a_mean, df.b_mean, rtol=0, atol=3*df.a_std)
                                                    & ~np.isclose(df.a_mean, df.c_mean, rtol=0, atol=3*df.a_std))]
print(len(df))

2019


#### Additional crystal prototype specific relaxations

Filter out bct -> fcc

In [43]:
relaxed_keys = []
bct_df = df[df.prototype == 'A6--In--bct'] 
fcc_df = df[df.prototype == 'A1--Cu--fcc']
for j, bct in bct_df.iterrows():
    fcc = fcc_df[(fcc_df.potential_id == bct.potential_id)]
    
    if (np.any(np.isclose(bct.c_mean,     fcc.c_mean,     rtol=0, atol=3*bct.c_std)) and 
        np.any(np.isclose(bct.E_coh_mean, fcc.E_coh_mean, rtol=0, atol=3*bct.E_coh_std))):
        relaxed_keys.append(bct.calc_key)

df = df[~df.calc_key.isin(relaxed_keys)]
print(len(df))

1892


Filter out L1<sub>0</sub> -> B2

In [44]:
relaxed_keys = []
l10_df = df[df.prototype == 'L1_0--AuCu'] 
b2_df = df[df.prototype == 'B2--CsCl']
for j, l10 in l10_df.iterrows():
    b2 = b2_df[l10.potential_id == b2_df.potential_id]
    
    if (np.any(np.isclose(l10.c_mean,     b2.c_mean,     rtol=0, atol=3*l10.c_std)) and 
        np.any(np.isclose(l10.E_coh_mean, b2.E_coh_mean, rtol=0, atol=3*l10.E_coh_std))):
        relaxed_keys.append(l10.calc_key)

df = df[~df.calc_key.isin(relaxed_keys)]
print(len(df))

1875


Build list of records where the structure relaxed to another structure

- __relaxed_df__ is all records that have relaxed to another structure

In [45]:
relaxed_df = processed_df[~processed_df.calc_key.isin(df.calc_key)]
print(len(relaxed_df))

1097


In [46]:
potentials = df.potential_id.unique()

In [47]:
i=6

In [48]:
print(potentials[i])
df[df.potential_id==potentials[i]].loc[:, ['calc_key', 
                                           'prototype', 
                                           'symbols', 
                                           'a_mean', 
                                           'b_mean', 
                                           'c_mean', 
                                           'a_std',
                                           'E_coh_mean']].sort_values('E_coh_mean')

2013--Smirnova-D-E--U-Mo-Xe


,calc_key,prototype,symbols,a_mean,b_mean,c_mean,a_std,E_coh_mean
2212,abd6e64d-b818-4e8e-8a1a-d2ed2bd255e2,A2--W--bcc,[Mo],3.147386,3.147386,3.147386,0.003618,-6.929364
1938,93201a81-e071-404d-b06e-41724a61a317,A4--C--dc,[Mo],5.046160,5.049899,5.029747,0.026075,-6.627571
166,0a237978-9df7-4af1-919e-58709d957dae,D0_3--BiF3,"[U, Mo]",6.458205,6.458205,6.458205,0.001312,-6.123954
601,25c1ba09-5145-4e93-98df-b7bce6997c75,A15--Cr3Si,"[Mo, U]",5.382534,5.382533,5.382534,0.000624,-4.649252
291,12a85b1a-4933-4a56-82e0-5b9213ef2d8c,B1--NaCl--rock-salt,"[Mo, U]",5.154742,5.154742,5.154742,0.000341,-4.540890
454,1be81e5c-eb0e-44b0-9192-83c54de11f59,A2--W--bcc,[U],3.552450,3.543352,3.549148,0.215843,-4.174843
389,17dbe875-d902-4d6a-b0c7-bc5d77ad355c,L1_2--AuCu3,"[U, Mo]",3.216536,6.482009,7.116315,32.334754,-4.048997
333,15173d02-c2fb-40b0-bb68-c0fb1965eeba,A6--In--bct,[U],3.269421,3.269454,6.038138,0.001566,-3.565898
816,3443851a-47dd-4382-80ed-b21c36e64c21,L2_1--AlCu2Mn--heusler,"[Xe, U, Mo]",8.406557,8.182662,7.172352,0.657099,-3.357060
217,0de48d1e-dccf-436c-81eb-327d5be5517c,C1--CaF2--fluorite,"[Xe, Mo]",7.948231,7.935395,7.904961,0.737908,-3.261834


In [49]:
with open('good_dynamic.txt', 'w') as f:
    for key in df.calc_key.tolist():
        f.write(key+'\n')    

In [50]:
with open('relaxed_dynamic.txt', 'w') as f:
    for key in relaxed_df.calc_key.tolist():
        f.write(key+'\n')    

### Load static calculations for comparison

In [25]:
static_recordstyle =    'calculation-system-relax'

static_df = []
for record in dbase.iget_records(style=static_recordstyle):
    try:
        static_df.append(record.todict())
    except:
        print('Failed to load', record.name)
static_df = pd.DataFrame(static_df)
print(str(len(static_df)) + ' records loaded')
print(static_df.keys())

76830 records loaded
Index([u'C', u'E_cohesive', u'calc_key', u'calc_script', u'error', u'final_a',
       u'final_b', u'final_c', u'initial_a', u'initial_b', u'initial_c',
       u'load', u'load_options', u'potential_id', u'potential_key',
       u'pressure_xx', u'pressure_yy', u'pressure_zz', u'prototype',
       u'sizemults', u'status', u'strainrange', u'symbols', u'temperature'],
      dtype='object')


In [33]:
static_df = static_df[pd.isnull(static_df.error)]
compositions = []
for i, calc in static_df.iterrows():
    compositions.append(comp_refine(calc.symbols, counts[calc.prototype]))
static_df = static_df.assign(composition=compositions)

In [34]:
static_df

,C,E_cohesive,calc_key,calc_script,error,final_a,final_b,final_c,initial_a,initial_b,...,pressure_xx,pressure_yy,pressure_zz,prototype,sizemults,status,strainrange,symbols,temperature,composition
0,[[ 1.50487311 0.91652909 0.91652909 0. ...,-4.280000e+00,0000620e-d3f3-430a-aa0c-1351b16c40f0,calc_refine_structure,NaN,2.866500,2.866500,2.866500,2.866443,2.866443,...,0.0,0.0,0.0,A2--W--bcc,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-06,[Fe],0.0,Fe
1,[[-0.16237024 0.80856755 0.80856755 0. ...,-4.098827e+00,000144f7-1ff9-4000-8b36-9de0fdfbe8a7,calc_refine_structure,NaN,3.273976,3.273976,3.273976,3.266332,3.266332,...,0.0,0.0,0.0,Ah--alpha-Po--sc,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-06,[Nb],0.0,Nb
2,[[ 0.74283495 0.19557465 0.19557465 0. ...,-1.472159e+00,0001736c-e26d-4d67-b3ea-665e163d5fd5,calc_LAMMPS_ELASTIC,NaN,5.498767,5.498767,5.498767,5.487437,5.487437,...,0.0,0.0,0.0,A15--Cr3Si,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-06,"[Mg, Mg]",0.0,Mg
3,[[ 0.5302645 0.3088983 0.3088983 0. ...,-3.892402e+00,0001da37-6af0-4a84-92a4-4129e20e2069,calc_refine_structure,NaN,3.658403,3.658403,3.658403,3.652793,3.652793,...,0.0,0.0,0.0,A1--Cu--fcc,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-06,[Fe],0.0,Fe
4,[[ 1.88315542 0.45144479 0.45144479 0. ...,-7.939595e+00,00021ede-2ea7-4a7c-ab53-0a3b6794638c,calc_refine_structure,NaN,2.740510,2.740510,2.740510,2.743719,2.743719,...,0.0,0.0,0.0,Ah--alpha-Po--sc,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-05,[Ta],0.0,Ta
5,[[ 8.78725378e+01 8.78727508e+01 8.787275...,-1.339536e+00,0002fbf9-33d4-40bc-bfc1-4bed81d81115,calc_LAMMPS_ELASTIC,NaN,11.500014,11.500014,11.500014,11.488980,11.488980,...,0.0,0.0,0.0,B3--ZnS--cubic-zinc-blende,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-05,"[Al, Al]",0.0,Al
6,[[ 4.62087967 5.11822006 5.11822006 0. ...,-9.007377e+00,0003d398-4ff1-49eb-bc10-7b0fe159b6d6,calc_LAMMPS_ELASTIC,NaN,3.474580,3.474580,3.474580,3.482234,3.482234,...,0.0,0.0,0.0,L1_2--AuCu3,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-05,"[Mn, Mn]",0.0,Mn
7,[[ 1.66632496 0.83731457 0.36199746 0. ...,-4.353372e+00,0004da75-7533-4d3c-8512-b963ffe8ebfb,calc_LAMMPS_ELASTIC,NaN,2.485886,4.305682,4.120416,2.502513,4.334479,...,0.0,0.0,0.0,A3--Mg--hcp,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-05,[Ni],0.0,Ni
8,[[ 1.40981879 0.92046077 0.92046077 0. ...,-3.864145e+00,00050fdf-d5e5-4982-9c13-e5b2497e0b17,calc_refine_structure,NaN,5.695602,5.695602,5.695602,5.686465,5.686465,...,0.0,0.0,0.0,L2_1--AlCu2Mn--heusler,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-07,"[Fe, Cu, Fe]",0.0,CuFe2
9,[[ 2.73498748e-01 3.20413961e-01 3.204139...,-2.304889e+00,00053492-8d9f-4b0d-a511-eb7dfb72feb3,calc_LAMMPS_ELASTIC,NaN,5.036954,5.036954,5.036954,5.036583,5.036583,...,0.0,0.0,0.0,L2_1--AlCu2Mn--heusler,"[[0, 3], [0, 3], [0, 3]]",NaN,1.000000e-08,"[Al, H, H]",0.0,AlH2


In [57]:
compare_df = []
for i, dynamic in df.iterrows():
    compare_dict = {}
    compare_dict['potential_id'] = dynamic.potential_id
    compare_dict['prototype'] =    dynamic.prototype
    compare_dict['composition'] =  dynamic.composition
    compare_dict['a_std'] =        dynamic.a_std
    compare_dict['b_std'] =        dynamic.b_std
    compare_dict['c_std'] =        dynamic.c_std
    compare_dict['E_coh_std'] =    dynamic.E_coh_std
    
    static = static_df[(static_df.potential_key == dynamic.potential_key) &
                       (static_df.prototype ==     dynamic.prototype) & 
                       (static_df.composition ==   dynamic.composition)]
    
    refine_structure = static[static.calc_script == 'calc_refine_structure']
    compare_dict['a_std_refine'] = np.sqrt(np.mean((refine_structure.final_a - dynamic.a_mean)**2))
    compare_dict['b_std_refine'] = np.sqrt(np.mean((refine_structure.final_b - dynamic.b_mean)**2))
    compare_dict['c_std_refine'] = np.sqrt(np.mean((refine_structure.final_c - dynamic.c_mean)**2))
    compare_dict['E_coh_std_refine'] = np.sqrt(np.mean((refine_structure.E_cohesive - dynamic.E_coh_mean)**2))
    
    lammps_elastic =   static[static.calc_script == 'calc_LAMMPS_ELASTIC']
    compare_dict['a_std_lammps'] = np.sqrt(np.mean((lammps_elastic.final_a - dynamic.a_mean)**2))
    compare_dict['b_std_lammps'] = np.sqrt(np.mean((lammps_elastic.final_b - dynamic.b_mean)**2))
    compare_dict['c_std_lammps'] = np.sqrt(np.mean((lammps_elastic.final_c - dynamic.c_mean)**2))    
    compare_dict['E_coh_std_lammps'] = np.sqrt(np.mean((lammps_elastic.E_cohesive - dynamic.E_coh_mean)**2))
    
    compare_df.append(compare_dict)
compare_df = pd.DataFrame(compare_df)
compare_df.to_csv('good_compare.csv', index=False)

In [56]:
compare_df.to_csv('good_compare.csv', index=False)

In [58]:
compare_df = []
for i, dynamic in df.iterrows():
    compare_dict = {}
    compare_dict['potential_id'] = dynamic.potential_id
    compare_dict['prototype'] =    dynamic.prototype
    compare_dict['composition'] =  dynamic.composition
    compare_dict['a_mean'] =       dynamic.a_mean
    compare_dict['b_mean'] =       dynamic.b_mean
    compare_dict['c_mean'] =       dynamic.c_mean
    compare_dict['E_coh_mean'] =   dynamic.E_coh_std    
    compare_dict['a_std'] =        dynamic.a_std
    compare_dict['b_std'] =        dynamic.b_std
    compare_dict['c_std'] =        dynamic.c_std
    compare_dict['E_coh_std'] =    dynamic.E_coh_std
    
    static = static_df[(static_df.potential_key == dynamic.potential_key) &
                       (static_df.prototype ==     dynamic.prototype) & 
                       (static_df.composition ==   dynamic.composition)]
    
    refine_structure = static[static.calc_script == 'calc_refine_structure']
    compare_dict['num_refine'] = len(refine_structure)
    compare_dict['num_refine_1_std'] = len(refine_structure[np.abs(refine_structure.final_a - dynamic.a_mean) <   dynamic.a_std])
    compare_dict['num_refine_2_std'] = len(refine_structure[np.abs(refine_structure.final_a - dynamic.a_mean) < 2*dynamic.a_std])
    compare_dict['num_refine_3_std'] = len(refine_structure[np.abs(refine_structure.final_a - dynamic.a_mean) < 3*dynamic.a_std])
    
    lammps_elastic =   static[static.calc_script == 'calc_LAMMPS_ELASTIC']
    compare_dict['num_lammps'] = len(lammps_elastic)
    compare_dict['num_lammps_1_std'] = len(lammps_elastic[np.abs(lammps_elastic.final_a - dynamic.a_mean) <   dynamic.a_std])
    compare_dict['num_lammps_2_std'] = len(lammps_elastic[np.abs(lammps_elastic.final_a - dynamic.a_mean) < 2*dynamic.a_std])
    compare_dict['num_lammps_3_std'] = len(lammps_elastic[np.abs(lammps_elastic.final_a - dynamic.a_mean) < 3*dynamic.a_std])
    
    compare_df.append(compare_dict)
compare_df = pd.DataFrame(compare_df)
compare_df.to_csv('good_compare2.csv', index=False)